In [1]:
import ISRAnalysis as an
import sys
import ROOT as rt

Welcome to JupyROOT 6.14/04


# unfolding

In [2]:
binDef = "FineCoarse" #
channel = "electron"
channel_postfix = "detector_preFSR"
year = "2016"
doSys = False

# Select regularization mode
#regularisation_mode = rt.TUnfold.kRegModeNone
#regularisation_mode = rt.TUnfold.kRegModeCurvature 
regularisation_mode = rt.TUnfold.kRegModeDerivative
# Select density mode
density_mode = rt.TUnfoldDensity.kDensityModeNone
ignoreBinZero = True

doInputStat = True
doRMStat = True
doModelUnc = False

# Redirect stdout nd stderr
sys.stderr = open("./log/stderr_"+channel+"_"+year+".txt", "w")
sys.stdout = open("./log/stdout_"+channel+"_"+year+".txt", "w")

detUnfold = an.ISRAnalysis("DetUNFOLD", year, channel, regularisation_mode, doInputStat, doRMStat, ignoreBinZero, "matrix", 
                           "Detector_Dressed_DRp1_Fiducial", binDef, channel_postfix, doModelUnc, "Pt")

In [3]:
#detUnfold.checkMatrixCond()

In [4]:
# Nominal
detUnfold.setInputHist(useMCInput=False, unfoldObj=None, dirName="Detector")

In [5]:
detUnfold.setUnfoldBkgs(dirName="Detector")

In [6]:
detUnfold.subFake() # Subract DY fake (out of fiducial volume at gen level), subUnfoldFake()

In [7]:
detUnfold.setInputHistUnfSys() # Set input for unfolding systematic

In [8]:
if doSys == True:
    
    PDF_sys_list = ["PDFerror{:0>3}".format(str(i)) for i in range(1,101)]
    
    if channel == "muon" :
            
        systematics = {
            "Type_2": ["IdSFUp", "IdSFDown", "IsoSFUp","IsoSFDown", "trgSFUp", "trgSFDown", "PUUp", "PUDown", 
                       "bvetoUp", "bvetoDown", 
                       "ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown", 
                       "AlphaSUp", "AlphaSDown",
                       "LepMomScaleUp","LepMomScaleDown"
                      ],
        }
        systematics["Type_2"] += PDF_sys_list
    
    if channel == "electron" :
        
        
            # 1. Input distribution 2. Background distribution 3. Response matrix
            # 
            # Type 1: Change input(data) and background histogram and response matrix (ex. Lepton scale)
            # Type 2: Change background histograms and response matrix (ex. Lepton ID SF)
            # Type 3: Change only background histotrams (Background normalization)
            # Type 4: Only response matrix changes (ex. Unfolding model systematic or statistics)
            # Type 5: Only unfolding setting changes (for unfolding mode, etc)
            #
            # Lets make a dictionary for all the systematics, and then select for the specific data year
            
            systematics = {
            "Type_2": ["IdSFUp", "IdSFDown", "recoSFUp","recoSFDown", "trgSFUp", "trgSFDown", "PUUp", "PUDown", 
                       "bvetoUp", "bvetoDown", 
                       "ScaleABUp", "ScaleABDown", "ScaleAUp", "ScaleADown", "ScaleBUp", "ScaleBDown", 
                       "AlphaSUp", "AlphaSDown",
                       #"L1PrefireUp", "L1PrefireDown",
                       "LepMomScaleUp","LepMomScaleDown","LepMomResUp","LepMomResDown"],
                       
            }
            systematics["Type_2"] += PDF_sys_list
            
    for sysType, sysNames in systematics.items():
        for sysName in sysNames:
            #print(sysName)
            detUnfold.setSystematics(sysType, sysName) # Currently, response matrix set here
            
            detUnfold.setInputHist(False, None, "Detector", sysType, sysName)
            detUnfold.setUnfoldBkgs("Detector", sysType, sysName) 
            detUnfold.subFake("Detector_DY_Fake", sysType, sysName)
            

In [9]:
#detUnfold.checkMatrixCond()

In [10]:
detUnfold.doUnfold(False) # nominal unfold

In [11]:
#detUnfold.closeOutFiles()

In [12]:
detUnfold.doAcceptance(isFSR=False, outName="DetEfficienyCorr", useMassBinned=False)
detUnfold.closeOutFiles()
#detUnfold.combineOutFiles()

#detUnfold_closure.closeOutFiles()
#detUnfold_closure.combineOutFiles()